<a href="https://colab.research.google.com/github/SriSatyaSatti/Adaptive-Model-Selection-for-Enhanced-Classification-Accuracy/blob/main/Song_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install -U gretel-client

In [ ]:
!pip install librosa
!pip install numpy
!pip install matplotlib
!pip install soundfile
!pip install scipy
!pip install psola

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 83.0 MB/s eta 0:00:00


In [ ]:
# Install pydub, gTTS, and IPython
!pip install pydub gtts
!pip install ipython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.2 MB/s eta 0:00:00


In [ ]:
!pip install gtts

In [ ]:
!pip install av
!pip install einops
!pip install flashy>=0.0.1
!pip install hydra-core>=1.1
!pip install hydra_colorlog
!pip install julius
!pip install num2words
!pip install numpy
!pip install sentencepiece
!pip install spacy==3.5.2
!pip install torch>=2.0.0
!pip install torchaudio>=2.0.0
!pip install huggingface_hub
!pip install tqdm
!pip install transformers>=4.31.0
!pip install xformers
!pip install demucs
!pip install librosa
!pip install gradio
!pip install torchmetrics
!pip install encodec
!pip install audiocraft
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 721.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for julius: filename=julius-0.2.7-py3-none-any.whl size=21870 sha256=5b1457799778736846126455bd3036c6b147a6de7bbabbc41242e01d2f145649
  Stored in directory: /root/.cache/pip/wheels/b9/b2/05/f883527ffcb7f2ead5438a2c23439aa0c881eaa9a4c80256f4
Successfully built julius
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=17644cbedd56bc6c3795db857115a465e72ea0d4a4786f2687b9ffdc1dfa5947
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
     ━━━━━━━━━━━━━━━━━━

In [ ]:
import json

import pandas as pd
from gretel_client import configure_session
from gretel_client.helpers import poll
from gretel_client.projects import create_or_get_unique_project, get_project

DATASET_PATH = 'https://gretel-public-website.s3.us-west-2.amazonaws.com/datasets/clean_lyrics.csv'
SEPARATOR = ','

pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Log into Gretel

configure_session(api_key="grtuea3f9221fb29ea53b7feaa55afef84851b98169ab3b1d8e1bcb8d238a095abf4", cache="yes", endpoint="https://api.gretel.cloud", validate=True, clear=True) #clear=True

project = create_or_get_unique_project(name="lyric-generator")

Caching Gretel config to disk.
Using endpoint https://api.gretel.cloud
Logged in as srisatyasatti@gmail.com ✅


In [ ]:
config = {
  "schema_version": 1,
  "models": [
    {
      "gpt_x": {
        "data_source": "__",
        "pretrained_model": "EleutherAI/gpt-neo-125M",
        "batch_size": 1,
        "epochs": 1,
        "weight_decay": 0.1,
        "warmup_steps": 100,
        "lr_scheduler": "cosine",
        "learning_rate": 0.00005
      }
    }
  ]
}

print(json.dumps(config, indent=2))

{
  "schema_version": 1,
  "models": [
    {
      "gpt_x": {
        "data_source": "__",
        "pretrained_model": "EleutherAI/gpt-neo-125M",
        "batch_size": 1,
        "epochs": 1,
        "weight_decay": 0.1,
        "warmup_steps": 100,
        "lr_scheduler": "cosine",
        "learning_rate": 5e-05
      }
    }
  ]
}


In [ ]:
def create_dataset(dataset_path: str) -> pd.DataFrame:
    """
    Combine intents and text into a single string to pass to GPT-X.
    """

    df = pd.read_csv(dataset_path)
    df['combined'] = df['Artist'] + SEPARATOR + df['Title'] + SEPARATOR + df['Clean Lyric'].apply(lambda x: str(x)[:256])
    df = df.dropna()
    return df

df = create_dataset(DATASET_PATH)
df[['combined']].to_csv('finetune.csv', index=False)
df

,Unnamed: 0,Artist,Title,Album,Year,Date,Lyric,Clean Lyric,combined
0,0.0,Dua Lipa,New Rules,Dua Lipa,2017.0,2017-06-02,one one one one one talkin' in my sleep at night makin' myself crazy out of my mind out of my mind wrote it down and read it out hopin' it would save me too many times too many times refrain my love he makes me feel like nobody else nobody else but my love he doesn't love me so i tell myself i tell myself pre one don't pick up the phone you know he's only callin' 'cause he's drunk and alone two don't let him in you'll have to kick him out again three don't be his friend you know you're gonna wake up in his bed in the morning and if you're under him you ain't gettin' over him i got new rules i count 'em i got new rules i count 'em i gotta tell them to myself i got new rules i count 'em i gotta tell them to myself i keep pushin' forwards but he keeps pullin' me backwards nowhere to turn no way nowhere to turn no now i'm standin' back from it i finally see the pattern i never learn i never learn refrain but my love he doesn't love me so i tell myself i tell myself i do i do i do pre one don't pick up the phone you know he's only callin' 'cause he's drunk and alone two don't let him in you'll have to kick him out again three don't be his friend you know you're gonna wake up in his bed in the morning and if you're under him you ain't gettin' over him i got new rules i count 'em i got new rules i count 'em i gotta tell them to myself i got new rules i count 'em i gotta tell them to myself practice makes perfect i'm still tryna learn it by heart i got new rules i count 'em eat sleep and breathe it rehearse and repeat it 'cause i i got new pre one don't pick up the phone yeah you know he's only callin' 'cause he's drunk and alone alone two don't let him in uhooh you'll have to kick him out again again three don't be his friend you know you're gonna wake up in his bed in the morning and if you're under him you ain't gettin' over him i got new rules i count 'em i got new rules i count 'em whoaooh whoaooh whoa i gotta tell them to myself i got new rules i count 'em baby you know i count 'em i gotta tell them to myself don't let him in don't let him in don't don't don't don't don't be his friend don't be his friend don't don't don't don't don't let him in don't let him in don't don't don't don't don't be his friend don't be his friend don't don't don't don't you're gettin' over him,one one one one one talkin' in my sleep at night makin' myself crazy out of my mind out of my mind wrote it down and read it out hopin' it would save me too many times too many times refrain my love he makes me feel like nobody else nobody else but my love he doesn't love me so i tell myself i tell myself pre one don't pick up the phone you know he's only callin' 'cause he's drunk and alone two don't let him in you'll have to kick him out again three don't be his friend you know you're gonna wake up in his bed in the morning and if you're under him you ain't gettin' over him i got new rules i count 'em i got new rules i count 'em i gotta tell them to myself i got new rules i count 'em i gotta tell them to myself i keep pushin' forwards but he keeps pullin' me backwards nowhere to turn no way nowhere to turn no now i'm standin' back from it i finally see the pattern i never learn i never learn refrain but my love he doesn't love me so i tell myself i tell myself i do i do i do pre one don't pick up the phone you know he's only callin' 'cause he's drunk and alone two don't let him in you'll have to kick him out again three don't be his friend you know you're gonna wake up in his bed in the morning and if you're under him you ain't gettin' over him i got new rules i count 'em i got new rules i count 'em i gotta tell them to myself i got new rules i count 'em i gotta tell them to myself practice makes perfect i'm still tryna learn it by heart i got new rules i count 'em eat sleep and breathe it rehearse and repeat it 'cause i i got new pre one don

In [ ]:
%%time

model = project.create_model_obj(model_config=config)
model.data_source = "finetune.csv"
model.name = "lyric-generator"
model.submit_cloud()

poll(model)

INFO: Starting poller
INFO: Status is created. Model creation has been queued.


{
    "uid": "65a0d7fc6d12da373062a31d",
    "guid": "model_2aqIptrOK6QsdRgceppEnrgY7II",
    "model_name": "lyric-generator",
    "model_key": "",
    "runner_mode": "cloud",
    "user_id": "659e65f753ab955ff4ea0e0c",
    "user_guid": "user_2al40NgEYqLgrpW1GK1nNSoSLHj",
    "billing_domain": null,
    "billing_domain_guid": null,
    "project_id": "659e66417a962af6574f2f63",
    "project_guid": "proj_2al49hijVVvvq0gDJEQCICaNE9w",
    "status_history": {
        "created": "2024-01-12T06:11:08.276258Z"
    },
    "last_modified": "2024-01-12T06:11:08.373593Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "annotations": null,
    "provenance": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/models/gpt_x@sha256:d46140f087c7e86285413c412882a4477083b9e584f4b210b81360ceda322961",
    "container_image_version": "2.10.101",
    "model_type": "gpt_x",
    "mo

INFO: Status is pending. A Gretel Cloud worker is being allocated to begin model creation.
INFO: Status is active. A worker has started creating your model!
2024-01-12T06:11:28.153571Z  Resolved revision for model
{
    "revision": "6cb0d322a3a484e99667e7cb240e22f1ac036b99",
    "model": "EleutherAI/gpt-neo-125M"
}
2024-01-12T06:11:28.157351Z  Starting GPT model training...
{
    "num_epochs": 1.0
}
2024-01-12T06:11:28.157786Z  Fine-tuning 'EleutherAI/gpt-neo-125M' with provided dataset!
2024-01-12T06:11:28.158046Z  Disclaimer: the chosen model may produce untrue and/or offensive content without warning. For more info, see https://docs.gretel.ai/reference/synthetics/models/gretel-gpt#limitations-and-biases
2024-01-12T06:11:28.158382Z  Downloading model from remote source. Depending on the size of the model, this may take a few minutes.
2024-01-12T06:11:33.347643Z  Successfully loaded model and tokenizer.
2024-01-12T06:12:34.198401Z  Training in progress, 47.1% complete (epoch 0.4714686

CPU times: user 1.23 s, sys: 164 ms, total: 1.4 s
Wall time: 3min 36s


2024-01-12T06:14:38.753197Z  Upload to Gretel Cloud is completed.


In [ ]:
# Example 2: Generate text with an optional "prompt" which is used to condition
# model generation.
user_input = input()
record_handler = model.create_record_handler_obj(
    params={"num_records": 5,
            "maximum_text_length": 128,
            "data_source": user_input}
)
record_handler.submit_cloud()
poll(record_handler)

df = pd.read_csv(record_handler.get_artifact_link("data"), compression='gzip')
df

Generate a song


INFO: Starting poller
INFO: Status is created. A job has been queued.


{
    "uid": "65a0d92fb4fcebf832ab79d0",
    "guid": "model_run_2aqJSXBoI4jYBybLNzXaYX9pzEn",
    "model_name": null,
    "model_key": "",
    "runner_mode": "cloud",
    "user_id": "659e65f753ab955ff4ea0e0c",
    "user_guid": "user_2al40NgEYqLgrpW1GK1nNSoSLHj",
    "billing_domain": null,
    "billing_domain_guid": null,
    "project_id": "659e66417a962af6574f2f63",
    "project_guid": "proj_2al49hijVVvvq0gDJEQCICaNE9w",
    "status_history": {
        "created": "2024-01-12T06:16:15.288000Z"
    },
    "last_modified": "2024-01-12T06:16:15.377000Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "annotations": null,
    "provenance": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/models/gpt_x@sha256:d46140f087c7e86285413c412882a4477083b9e584f4b210b81360ceda322961",
    "container_image_version": "2.10.101",
    "model_id": "65a0d7fc6d12da373062a31d",

INFO: Status is pending. A Gretel Cloud worker is being allocated
INFO: Status is active. A worker has started!
2024-01-12T06:16:22.564012Z  Loading model to worker
2024-01-12T06:16:46.716236Z  Sampling 5 records using auto prompting.
2024-01-12T06:16:46.717000Z  Using device 'cuda'
2024-01-12T06:16:49.306394Z  Generating records...
{
    "num_records": 5
}
2024-01-12T06:16:52.438032Z  Successfully generated 5 records
2024-01-12T06:16:52.449577Z  Uploading artifacts to Gretel Cloud...
2024-01-12T06:16:52.806143Z  Upload to Gretel Cloud is completed.


,combined
0,"Drake,Lift Me Up (AJ Love),i don't need the lift cause i know you do it cause i don't wanna hurt you no matter what you do cause you got me caught in the act cause you wanna hurt me i'm a sucker for the lift but i'm the one you're holding on to you i wanna let you know that you can't let me down no matter what i do yeah i've been waiting for you i've been waiting for you"
1,"Drake,Gravity,drake drake i got him like a couple of things drake yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah drake it's the best i've ever been this ******* ******* drake drake it's the best i've ever been this ******* ******* drake drake i'm the ***** drake drake yeah yeah"
2,"Dua Lipa,​The Girl I Was,i feel the pressure that i’ve been working on i think i can handle it i’ll try to get better but i just don't see how we could do it with the energy we have right now but i don't see how we could do it with the energy we have right now but i don't see how we could do it with the energy we have right now but i don't see how we could do it with the energy we have right now but i don't see how we could do it with the energy we have right now but i don't see"
3,"Katy Perry,Pleasure,i wanna feel this way i wanna feel this way i wanna feel this way i wanna feel this way i wanna feel this way i wanna feel this way i wanna feel this way"
4,"Rihanna,You’re a Good Girl,oh my god i got this baby boy baby boy baby boy baby boy baby boy baby boy baby boy baby boy baby boy baby boy baby boy baby boy baby boy baby boy baby baby baby baby baby boy baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby baby"


In [ ]:
from gtts import gTTS
from IPython.display import Audio
import os

def generate_voice_output(output_text, output_directory="output"):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Create a gTTS object
    tts = gTTS(text=output_text, lang='en')

    # Specify the file path including the output directory
    file_path = os.path.join(output_directory, "output.mp3")

    # Save the speech as an audio file
    tts.save(file_path)

    # Display the audio file
    #return Audio(file_path, autoplay=True)

# Example usage with the output stored in the "audio_files" directory
text_to_speak = df.combined[0]+df.combined[1]+df.combined[2]+df.combined[3]
generate_voice_output(text_to_speak, output_directory="audio_files")


In [ ]:
from functools import partial
from pathlib import Path
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import scipy.signal as sig
import psola

SEMITONES_IN_OCTAVE = 12

def degrees_from(scale: str):
    degrees = librosa.key_to_degrees(scale)
    degrees = np.concatenate((degrees, [degrees[0] + SEMITONES_IN_OCTAVE]))
    return degrees

def closest_pitch(f0):
    midi_note = np.around(librosa.hz_to_midi(f0))
    nan_indices = np.isnan(f0)
    midi_note[nan_indices] = np.nan
    return librosa.midi_to_hz(midi_note)

def closest_pitch_from_scale(f0, scale):
    if np.isnan(f0):
        return np.nan
    degrees = degrees_from(scale)
    midi_note = librosa.hz_to_midi(f0)
    degree = midi_note % SEMITONES_IN_OCTAVE
    degree_id = np.argmin(np.abs(degrees - degree))
    degree_difference = degree - degrees[degree_id]
    midi_note -= degree_difference
    return librosa.midi_to_hz(midi_note)

def aclosest_pitch_from_scale(f0, scale):
    sanitized_pitch = np.zeros_like(f0)
    for i in np.arange(f0.shape[0]):
        sanitized_pitch[i] = closest_pitch_from_scale(f0[i], scale)
    smoothed_sanitized_pitch = sig.medfilt(sanitized_pitch, kernel_size=11)
    smoothed_sanitized_pitch[np.isnan(smoothed_sanitized_pitch)] = \
        sanitized_pitch[np.isnan(smoothed_sanitized_pitch)]
    return smoothed_sanitized_pitch

def autotune(audio, sr, correction_function, plot=False):
    frame_length = 2048
    hop_length = frame_length // 4
    fmin = librosa.note_to_hz('C2')
    fmax = librosa.note_to_hz('C7')

    f0, voiced_flag, voiced_probabilities = librosa.pyin(audio,
                                                         frame_length=frame_length,
                                                         hop_length=hop_length,
                                                         sr=sr,
                                                         fmin=fmin,
                                                         fmax=fmax)

    corrected_f0 = correction_function(f0)

    if plot:
        stft = librosa.stft(audio, n_fft=frame_length, hop_length=hop_length)
        time_points = librosa.times_like(stft, sr=sr, hop_length=hop_length)
        log_stft = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
        fig, ax = plt.subplots()
        img = librosa.display.specshow(log_stft, x_axis='time', y_axis='log', ax=ax, sr=sr, hop_length=hop_length, fmin=fmin, fmax=fmax)
        fig.colorbar(img, ax=ax, format="%+2.f dB")
        ax.plot(time_points, f0, label='original pitch', color='cyan', linewidth=2)
        ax.plot(time_points, corrected_f0, label='corrected pitch', color='orange', linewidth=1)
        ax.legend(loc='upper right')
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [M:SS]')
        plt.savefig('pitch_correction.png', dpi=300, bbox_inches='tight')

    return psola.vocode(audio, sample_rate=int(sr), target_pitch=corrected_f0, fmin=fmin, fmax=fmax)

def main():
    # Specify the path to your local MP3 file
    mp3_filepath = "/content/audio_files/output.mp3"

    # Load the audio file.
    y, sr = librosa.load(mp3_filepath, sr=None, mono=False)

    pitch_corrected_y = None  # Initialize pitch_corrected_y

    if y.ndim > 1:
        y = y[0, :]

    correction_function = closest_pitch

    # Specify the output directory
    output_directory = Path("path/output_directory")
    output_directory.mkdir(parents=True, exist_ok=True)

    # Assign pitch_corrected_y
    pitch_corrected_y = autotune(y, sr, correction_function, plot=False)

    # Print the shape of pitch_corrected_y for debugging
    print("Shape of pitch_corrected_y:", pitch_corrected_y.shape)

    # Write the corrected audio to an output file.
    output_filepath = output_directory / (Path(mp3_filepath).stem + '_pitch_corrected' + Path(mp3_filepath).suffix)
    sf.write(str(output_filepath), pitch_corrected_y, sr)

    print(f"Pitch-corrected audio saved to: {output_filepath}")

if __name__ == '__main__':
    main()




Shape of pitch_corrected_y: (2767680,)
Pitch-corrected audio saved to: path/output_directory/output_pitch_corrected.mp3


In [ ]:
from audiocraft.models import MusicGen
import torch
import torchaudio
import os

def load_model():
    model = MusicGen.get_pretrained('facebook/musicgen-small')
    return model

def generate_music_tensors(description, duration: int):
    print("Description: ", description)
    print("Duration: ", duration)
    model = load_model()

    model.set_generation_params(
        use_sampling=True,
        top_k=250,
        duration=duration
    )

    output = model.generate(
        descriptions=[description],
        progress=True,
        return_tokens=True
    )

    return output[0]

def save_audio(samples: torch.Tensor, save_path="audio_output/"):
    print("Samples (inside function): ", samples)
    sample_rate = 32000
    assert samples.dim() == 2 or samples.dim() == 3

    samples = samples.detach().cpu()
    if samples.dim() == 2:
        samples = samples[None, ...]

    # Create the output directory if it doesn't exist
    os.makedirs(save_path, exist_ok=True)

    for idx, audio in enumerate(samples):
        audio_path = os.path.join(save_path, f"audio.mp3")
        torchaudio.save(audio_path, audio, sample_rate)

    print(f"Audio saved to: {save_path}")

def main(description, duration):
    model = load_model()

    model.set_generation_params(
        use_sampling=True,
        top_k=250,
        duration=duration
    )

    output = model.generate(
        descriptions=[description],
        progress=True,
        return_tokens=True
    )

    save_audio(output[0])

description_input = user_input
duration_input = 30

main(description_input, duration_input)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


state_dict.bin:   0%|          | 0.00/841M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

compression_state_dict.bin:   0%|          | 0.00/236M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Samples (inside function):  tensor([[[ 0.0233,  0.0198,  0.0489,  ...,  0.0012, -0.0111, -0.0111]]],
       device='cuda:0')
Audio saved to: audio_output/


In [ ]:
from pydub import AudioSegment
from pydub.playback import play


def play_two_audios_simultaneously(file1_path, file2_path, output_path):
    # Load the audio files
    audio1 = AudioSegment.from_file(file1_path)
    audio2 = AudioSegment.from_file(file2_path)

    # Determine the duration of each audio file
    duration1 = len(audio1)
    duration2 = len(audio2)

    # Choose the longer duration for looping
    longer_duration = max(duration1, duration2)

    # Calculate the number of loops needed for the shorter file
    loops = int(longer_duration / min(duration1, duration2))

    # Create an empty audio segment to store the merged audio
    merged_audio = AudioSegment.silent(duration=0)

    # Concatenate the shorter audio file to the merged audio for the specified loops
    for _ in range(loops):
        merged_audio += audio1 if duration1 < duration2 else audio2

    # Concatenate the remaining part of the longer audio file
    if duration1 < duration2:
        merged_audio += audio2[:longer_duration - len(merged_audio)]
    else:
        merged_audio += audio1[:longer_duration - len(merged_audio)]

    # Export the merged audio to a file
    merged_audio.export(output_path, format="mp3")





# Example usage with local file paths
file1_path = "/content/path/output_directory/output_pitch_corrected.mp3"
file2_path = "/content/audio_output/audio.mp3"
output_path = "/content/path/merged_audio.mp3"

play_two_audios_simultaneously(file1_path, file2_path, output_path)

In [ ]:
from IPython.display import Audio, display

def play_two_local_audios(file1_path, file2_path):
    # Play both audio files simultaneously
    audio1 = Audio(file1_path, autoplay=True)
    audio2 = Audio(file2_path, autoplay=True)

    display(audio1, audio2)

# Example usage with local file paths
file1_path = "/content/path/merged_audio.mp3"
file2_path = "/content/path/output_directory/output_pitch_corrected.mp3"
play_two_local_audios(file1_path, file2_path)